#### **First Step**: Write the queries for the PostgreSQL database based on the data.


Task:

- Identify the data types to write the database `schema.sql`
- Transfer the data from the CSV to a query that allows me to insert all the `seed_data.sql`


In [45]:
import pandas as pd
import numpy as np

import logging

logging.basicConfig(level=logging.INFO)


def infer_sql_type(dtype):
    """Infer SQL type from a pandas dtype."""
    logging.info(f"Infering SQL type for {dtype}")
    if np.issubdtype(dtype, np.integer):
        return "INTEGER"
    elif np.issubdtype(dtype, np.floating):
        return "FLOAT"
    elif np.issubdtype(dtype, np.bool_):
        return "BOOLEAN"
    elif np.issubdtype(dtype, np.datetime64):
        return "TIMESTAMP"
    else:
        return "TEXT"


def generate_schema(df, table_name='my_table'):
    """Generate the schema based on the DataFrame."""
    logging.info(f"Generating schema for {table_name}")
    columns = []
    for col in df.columns:
        col_type = infer_sql_type(df[col].dtype)
        columns.append(f'"{col}" {col_type}')
    schema = f"CREATE TABLE IF NOT EXISTS {table_name} (\n" + ",\n".join(columns) + "\n);"
    return schema


def generate_seed_data(df, table_name='my_table'):
    """Generate seed data SQL statements."""
    logging.info(f"Generating seed data for {table_name}")
    insert_statements = []
    for _, row in df.iterrows():
        values = []
        for value in row:
            if pd.isna(value):
                values.append("NULL")
            elif isinstance(value, str):
                values.append(f"'{value.replace('\'', '\'\'')}'")
            else:
                values.append(str(value))
        insert_statements.append(
            f"INSERT INTO {table_name} VALUES (" + ", ".join(values) + ");")
    return "\n".join(insert_statements)


def main(csv_file, schema_file, seed_file, table_name, delimiter):
    # Load the CSV file into a DataFrame
    logging.info(f"Loading CSV file {csv_file}")
    df = pd.read_csv(csv_file, delimiter=delimiter)
    # Remove spaces from column names
    df.columns = df.columns.str.replace(' ', '_')

    # Generate the SQL schema
    schema = generate_schema(df, table_name)
    with open(schema_file, 'w') as f:
        f.write(schema)

    # Generate the SQL seed data
    seed_data = generate_seed_data(df, table_name)
    with open(seed_file, 'w') as f:
        f.write(seed_data)


if __name__ == "__main__":
    main(csv_file='../data/raw/candidates.csv', schema_file='../sql/schema.sql',
         seed_file='../sql/seed_data.sql', table_name='candidates', delimiter=';')

2024-08-15 01:10:42,786 - Loading CSV file ../data/raw/candidates.csv
2024-08-15 01:10:42,875 - Generating schema for candidates
2024-08-15 01:10:42,876 - Infering SQL type for object
2024-08-15 01:10:42,877 - Infering SQL type for object
2024-08-15 01:10:42,878 - Infering SQL type for object
2024-08-15 01:10:42,878 - Infering SQL type for object
2024-08-15 01:10:42,879 - Infering SQL type for object
2024-08-15 01:10:42,879 - Infering SQL type for int64
2024-08-15 01:10:42,879 - Infering SQL type for object
2024-08-15 01:10:42,880 - Infering SQL type for object
2024-08-15 01:10:42,881 - Infering SQL type for int64
2024-08-15 01:10:42,881 - Infering SQL type for int64
2024-08-15 01:10:42,881 - Generating seed data for candidates


---


#### **Second Step**: Upload data to database


Task: 

* Import db class to use connector
* Establish connection and execute the queries to create the schema and send the data.
* Validate that the table has been created and that all records have been loaded.

In [4]:
import sys
import os

# Add the 'src' folder to sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from connections.db import DB

db = DB()

In [5]:
# Create schema
db.execute("../sql/schema.sql")

2024-08-15 01:34:42,068 - ✔ Connected to database
2024-08-15 01:34:42,606 - ✔ Query executed
2024-08-15 01:34:42,607 - ✔ Cursor closed
2024-08-15 01:34:42,607 - ✔ Connection closed


In [6]:
# Seed data
db.execute("../sql/seed_data.sql")

2024-08-15 01:34:44,245 - ✔ Connected to database
2024-08-15 01:34:59,033 - ✔ Query executed
2024-08-15 01:34:59,033 - ✔ Cursor closed
2024-08-15 01:34:59,033 - ✔ Connection closed


In [7]:
# View tables
db.execute("../sql/queries/001_view_tables.sql")

2024-08-15 01:34:59,941 - ✔ Connected to database
2024-08-15 01:35:00,544 - ✔ Query executed
2024-08-15 01:35:00,544 - ✔ Cursor closed
2024-08-15 01:35:00,546 - ✔ Connection closed


[('candidates',)]

In [8]:
# View size of tables
db.execute("../sql/queries/003_view_tables_sizes.sql")

2024-08-15 01:35:04,070 - ✔ Connected to database
2024-08-15 01:35:04,457 - ✔ Query executed
2024-08-15 01:35:04,457 - ✔ Cursor closed
2024-08-15 01:35:04,457 - ✔ Connection closed


[('public.candidates', 50000)]

-----

#### **Results**: 

* Created the query to define the database schema based on the data.
* Created the query to insert the data seed to the database. 
* Connection established with the database. 
* Creation of table with the defined schema. 
* Upload data to the table.

----